## SparkSession and SparkContext

SparkSession is accessible via the `spark` object and SparkContext is accessible via the `spark.sparkContext` object.

SparkContext gives access to lower-level API (RDDs).

Explore the methods available by each object using the autocomplete function.

In [0]:
# Spark Session object
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

In [0]:
# Spark Context object
spark.sparkContext

SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

In [0]:
# or simply
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

## RDDs vs DataFrames

The following example illustrates the expressiveness of the RDD and DataFrame APIs.

### RDD

In [0]:
# input synthethic data
# array of tuples (name, age)
data = [("Brooke", 20), ("Denny", 20), ("Jules", 30), ("TD", 35), ("Brooke", 25)]

dataRDD = sc.parallelize(data)

dataRDD

Out[1]: ParallelCollectionRDD[0] at readRDDFromInputStream at PythonRDD.scala:435

Q1: Has dataRDD been loaded into memory?

Q2: What does the code below do?

In [0]:
# operate on the RDD
agesRDD = (
    dataRDD
    .map(lambda x: (x[0], (x[1], 1)))
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
    .map(lambda x: (x[0], x[1][0]/x[1][1]))
)

agesRDD

Out[2]: PythonRDD[5] at RDD at PythonRDD.scala:58

In [0]:
# Inspect the resulting DAG using the UI tool
agesRDD.collect()

Out[3]: [('Brooke', 22.5), ('Denny', 20.0), ('TD', 35.0), ('Jules', 30.0)]

In [0]:
# Try again
# Inspect the resulting DAG using the UI tool
# What is the difference?
agesRDD.collect()

Out[4]: [('Brooke', 22.5), ('Denny', 20.0), ('TD', 35.0), ('Jules', 30.0)]

A skipped stage indicates that the data has been fetched from cache.

Whenever there is shuffling involved (group/partition by) Spark automatically caches generated data (credit https://stackoverflow.com/a/34581152/11217701)

### DataFrame

Same as above but now using the DataFrame API.

In [0]:
import pyspark.sql.functions as fn

In [0]:
# create Dataframe
data_df = spark.createDataFrame(data, schema=["name", "age"])
data_df

Out[6]: DataFrame[name: string, age: bigint]

In [0]:
data_df.show()

+------+---+
|  name|age|
+------+---+
|Brooke| 20|
| Denny| 20|
| Jules| 30|
|    TD| 35|
|Brooke| 25|
+------+---+



In [0]:
avg_df = data_df.groupBy("name").agg(fn.avg("age"))
avg_df

Out[8]: DataFrame[name: string, avg(age): double]

In [0]:
avg_df.show()

+------+--------+
|  name|avg(age)|
+------+--------+
| Jules|    30.0|
|    TD|    35.0|
|Brooke|    22.5|
| Denny|    20.0|
+------+--------+



**Q:** Which code is more readable?